In [0]:
EVENT_HUB_CONNECTION_STR = dbutils.secrets.get(scope = "eventhubs", key = "EVENT_HUB_CONNECTION_STR")
"""
eHconf = {
    'eventhubs.connectionString': sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(EVENT_HUB_CONNECTION_STR),
    'eventhubs.eventHubName': 'liveprices',
    "startingPosition": "@earliest"
}
"""

EH_NAMESPACE                    = 'predictstockpriceskafka'
EH_NAME                         = 'liveprices'

EH_CONN_STR                     = EVENT_HUB_CONNECTION_STR

KAFKA_OPTIONS = {
  "kafka.bootstrap.servers"  : f"{EH_NAMESPACE}.servicebus.windows.net:9093",
  "subscribe"                : EH_NAME,
  "kafka.sasl.mechanism"     : "PLAIN",
  "kafka.security.protocol"  : "SASL_SSL",
  "kafka.sasl.jaas.config"   : f"kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"$ConnectionString\" password=\"{EH_CONN_STR}\";",
  "startingOffsets"          : "earliest",
}

In [0]:
from pyspark.sql.functions import *

df = spark.readStream \
    .format("kafka") \
    .options(**KAFKA_OPTIONS) \
    .load()

df.display()


In [0]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("ticker", StringType()),
    StructField("price", StringType()),
    StructField("time", StringType())
])

parsed_df = df.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json(col("json_str"), schema).alias("data")) \
    .select("data.*")

#parsed_df = df.selectExpr("CAST(value AS STRING) as raw_value").display()

display(parsed_df)

In [0]:
spark.conf.set(
    "fs.azure.account.key.stocksstorage.dfs.core.windows.net",
    dbutils.secrets.get(scope="blobstorage", key="accesskey"))


parsed_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "abfss://plstocks@stocksstorage.dfs.core.windows.net/checkpoints/bronze") \
    .table("plstocks.bronze_prices_raw_stream")

In [0]:
spark.sql("SHOW TABLES in plstocks").show()